## 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
from tqdm import tqdm_notebook
result = pd.DataFrame()
for i in tqdm_notebook(range(599614, 599620)):
    URL = "https://www1.president.go.kr/petitions/" + str(i)
    
    response = requests.get(URL)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h3', class_='petitionsView_title')
    count = soup.find('span', class_='counter')
    
    for content in soup.select('div.petitionsView_write > div.View_write'):
        content
        
        a = []
        for tag in soup.select('ul.petitionsView_info_list > li'):
            a.append(tag.contents[1])
            
        if len(a) != 0:
            df1 = pd.DataFrame({'start':[a[1]],
                               'end' : [a[2]],
                               'category': [a[0]],
                               'count' : [count.text],
                               'title' : [title.text],
                               'content': [content.text.strip()[0:13000]]
                               })
            result = pd.concat([result, df1])
            
            result.index = np.arange(len(result))
            
        if i % 60 == 0:
            print('sleep 90seconds Count:' + str(i)
                 +", Local Time : ")
            +time.strftime('%y-%m-%d', time.localtime(time.time()))
            + " "+ time.strftime('%X', time.localtime(time.time()))
            + ", Data Length:" + str(len(result))
        time.sleep(90)

<ipython-input-2-bf8b9c4eba3a>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(599614, 599620)):


  0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
print(result.shape)

(7, 6)


In [4]:
df = result
df.head()

,start,end,category,count,title,content
0,2021-07-06,2021-08-05,인권/성평등,"213,854",학교 폭력으로 인해 생을 마감한 아들의 억울함을 풀어주세요.,
1,2021-07-06,2021-08-05,인권/성평등,"213,854",학교 폭력으로 인해 생을 마감한 아들의 억울함을 풀어주세요.,<학교 폭력으로 인해 생을 마감한 아들의 억울함과 가해자의 처벌 그리고 학교 폭력 ...
2,2021-07-06,2021-08-05,문화/예술/체육/언론,"2,764",학생선수들의 꿈을 지켜주세요,펜싱선수 **의 꿈을 지켜주세요 짓밟지말아주세요\r\n학생선수로서 공부와 운동에 최...
3,2021-07-06,2021-08-05,보건복지,568,질병 관리청장님은 백신과의 인과성과 부작용을 해명하고 보상안을 발표하여 주세요,백신으로 인한 부작용 이상 반응 사례자들이 갈 수록 늘어나고 있습니다. 원인을 알수...
4,2021-07-06,2021-08-05,육아/교육,"1,047",유치원 cctv 설치 여부 논란에 관한 유치원교사의 개인적인 의견입니다.,글을 쓰기에 앞서 이 글은 저의 개인적인 의견임을 먼저 밝힙니다.\n\r\n안녕하세...


In [5]:
df.loc[1]['content']

'<학교 폭력으로 인해 생을 마감한 아들의 억울함과 가해자의 처벌 그리고 학교 폭력 없는 세상이 올 수 있도록 도와 주세요>\n\r\n6월 29일 화요일 환하게 웃는 표정으로 손을 흔들며 학교에 간다던 아들이 학교에 가지 않고 인근 산으로 가 스스로 생을 마감했습니다. \r\n장례를 치르던 중 교실에서 폭행을 당하고 있는 아들의 모습이 담긴 영상을 제보받고 이유를 알게 되었습니다. 수년간의 학교폭력을 견디다 못해 스스로 선택한 마지막 길이였다는 것을...\n\r\n아들이 매일 웃으며 저의 퇴근길을 반겨주었는데... 학교에서 친구들과 잘 지낸다고 항상 씩씩하게 말하던 녀석인데. 속으로 그 큰 고통을 혼자 참고 견디고 있었다는 것을 생각하니 아비로써 너무나 가슴이 아픕니다.\n\r\n지금부터 시작입니다. 학교 폭력을 가한 가해 학생들이 엄중한 처벌을 받을 수 있도록, 국민 여러분 저희가 지지치 않고 싸울 수 있도록 옆에서 함께 해주십시오\r\n우리 아들의 억울함을 풀고, 학교 폭력이 없는 세상이 오도록 끝까지 도와주십시오.\n\n [본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]'

## 데이터 전처리

In [6]:
import re
# 공백문자를 모두 공백으로 치환한다.
def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

# 한글과 숫자를 제외한 모든 문자 제거 -> 특수 문자 제거
def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣0-9]+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [7]:
df.loc[1]['content']

' 학교 폭력으로 인해 생을 마감한 아들의 억울함과 가해자의 처벌 그리고 학교 폭력 없는 세상이 올 수 있도록 도와 주세요    6월 29일 화요일 환하게 웃는 표정으로 손을 흔들며 학교에 간다던 아들이 학교에 가지 않고 인근 산으로 가 스스로 생을 마감했습니다    장례를 치르던 중 교실에서 폭행을 당하고 있는 아들의 모습이 담긴 영상을 제보받고 이유를 알게 되었습니다  수년간의 학교폭력을 견디다 못해 스스로 선택한 마지막 길이였다는 것을    아들이 매일 웃으며 저의 퇴근길을 반겨주었는데  학교에서 친구들과 잘 지낸다고 항상 씩씩하게 말하던 녀석인데  속으로 그 큰 고통을 혼자 참고 견디고 있었다는 것을 생각하니 아비로써 너무나 가슴이 아픕니다    지금부터 시작입니다  학교 폭력을 가한 가해 학생들이 엄중한 처벌을 받을 수 있도록  국민 여러분 저희가 지지치 않고 싸울 수 있도록 옆에서 함께 해주십시오  우리 아들의 억울함을 풀고  학교 폭력이 없는 세상이 오도록 끝까지 도와주십시오     본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다 '

## 토큰나이징 

In [8]:
!pip install konlpy

In [9]:
from konlpy.tag import Okt

In [10]:
okt = Okt()

df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

SystemError: java.nio.file.InvalidPathException: Illegal char <*> at index 60: C:\Users\CPB06GameN\anaconda3\Lib\site-packages\konlpy\java\*

In [ ]:
df['token_final'] = df.title_token + df.content_token

df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))
print(df.dtypes)

In [ ]:
df['label'] = df['count'].apply(lambda x: 'yes' if x>=1000 else 'no')

In [ ]:
df_drop = df[['token_final', 'label']]

In [ ]:
from gensim.models import Word2Vec

In [ ]:
emb_model = Word2Vec(df_drop['token_final'],
                     sg = 1,
                     size = 100,
                     window = 2,
                     min_count = 1,
                     workers = 4

)

print(emb_model)
model_result = emb_model.wv.most_similar('폭력')
print(model_result)

In [ ]:
from gensim.models import KeyedVectors
emb_model.wv.save_word2vec_format('/content/data')
loaded_model = KeyedVectors.load_word2vec_format('/content/data')
modle_result = loaded_model.most_similar('폭력')
print(modle_result )

In [ ]:
from numpy.random import RandomState
rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('/content/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('/content/val.csv', index=False, encoding='utf-8-sig')

In [ ]:
import torchtext
from torchtext.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize= tokenizer)
LABEL = Field(sequential = False)